In [120]:
import warnings
import joblib
import pandas as pd
import numpy as np
from pandas import read_csv
from sklearn.preprocessing import OrdinalEncoder
import seaborn as sns
from matplotlib import pyplot
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import f_regression
from sklearn.metrics import mean_squared_error,mean_absolute_error, mean_absolute_percentage_error, r2_score
#%matplotlib inline
warnings.filterwarnings('ignore')

In [121]:
#load your data into the dataframe
my_data = read_csv('2005-19_Local_Authority_CO2_emissions(1).csv')

In [116]:
my_data.head()

,Country,Country Code,Region,Region Code,Second Tier Authority,Local Authority,Local Authority Code,Calendar Year,LA CO2 Sector,LA CO2 Sub-sector,Territorial emissions (kt CO2),Emissions within the scope of influence of LAs (kt CO2),Mid-year Population (thousands),Area (km2)
0,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Commercial,Commercial Electricity,77.193230,77.193230,90.457,98.3466
1,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Commercial,Commercial Gas,16.689241,16.689241,90.457,98.3466
2,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Commercial,Commercial 'Other Fuels',0.198111,0.198111,90.457,98.3466
3,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Domestic,Domestic Electricity,80.639845,80.639845,90.457,98.3466
4,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Domestic,Domestic Gas,131.383713,131.383713,90.457,98.3466


In [95]:
print(my_data.shape)

(129388, 14)


In [96]:
my_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129388 entries, 0 to 129387
Data columns (total 14 columns):
 #   Column                                                   Non-Null Count   Dtype  
---  ------                                                   --------------   -----  
 0   Country                                                  129235 non-null  object 
 1   Country Code                                             129235 non-null  object 
 2   Region                                                   129235 non-null  object 
 3   Region Code                                              129235 non-null  object 
 4   Second Tier Authority                                    129235 non-null  object 
 5   Local Authority                                          129388 non-null  object 
 6   Local Authority Code                                     129265 non-null  object 
 7   Calendar Year                                            129388 non-null  int64  
 8   LA CO2 Sector 

In [30]:
#check our data for missing value
my_data = my_data[my_data.isna().any(axis=1)]
print('\n These are the missing data \n', my_data)


 These are the missing data 
        Country Country Code Region Region Code Second Tier Authority  \
110673     NaN          NaN    NaN         NaN                   NaN   
110674     NaN          NaN    NaN         NaN                   NaN   
110675     NaN          NaN    NaN         NaN                   NaN   
110676     NaN          NaN    NaN         NaN                   NaN   
110677     NaN          NaN    NaN         NaN                   NaN   
...        ...          ...    ...         ...                   ...   
129383     NaN          NaN    NaN         NaN                   NaN   
129384     NaN          NaN    NaN         NaN                   NaN   
129385     NaN          NaN    NaN         NaN                   NaN   
129386     NaN          NaN    NaN         NaN                   NaN   
129387     NaN          NaN    NaN         NaN                   NaN   

                   Local Authority Local Authority Code  Calendar Year  \
110673  Unallocated electricit

In [81]:
my_data.rename(columns={'Territorial emissions (kt CO2)':'CO2_emission'})

,Country,Country Code,Region,Region Code,Second Tier Authority,Local Authority,Local Authority Code,Calendar Year,LA CO2 Sector,LA CO2 Sub-sector,CO2_emission,Emissions within the scope of influence of LAs (kt CO2),Mid-year Population (thousands),Area (km2)
0,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Commercial,Commercial Electricity,7.719323e+01,77.193230,90.457,98.3466
1,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Commercial,Commercial Gas,1.668924e+01,16.689241,90.457,98.3466
2,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Commercial,Commercial 'Other Fuels',1.981106e-01,0.198111,90.457,98.3466
3,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Domestic,Domestic Electricity,8.063985e+01,80.639845,90.457,98.3466
4,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Domestic,Domestic Gas,1.313837e+02,131.383713,90.457,98.3466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129383,NaN,NaN,NaN,NaN,NaN,Unallocated consumption,NaN,2019,Industry,Industry Gas,2.574098e+02,257.409829,NaN,NaN
129384,NaN,NaN,NaN,NaN,NaN,Unallocated consumption,NaN,2019,LULUCF,LULUCF Net Emissions: Forest land,-3.270000e-11,0.000000,NaN,NaN
129385,NaN,NaN,NaN,NaN,NaN,Unallocated consumption,NaN,2019,LULUCF,LULUCF Net Emissions: Cropland,1.820000e-11,0.000000,NaN,NaN
129386,NaN,NaN,NaN,NaN,NaN,Unallocated consumption,NaN,2019,LULUCF,LULUCF Net Emissions: Grassland,8.640000e-12,0.000000,NaN,NaN


In [122]:
# btc=my_data[my_data['symbol']=='BTC']#Extracting the bitcoin data from the dataframe
# btc
Eng_land =my_data[my_data['Country']=='England']#Extracting England data from the dataframe
Eng_land

,Country,Country Code,Region,Region Code,Second Tier Authority,Local Authority,Local Authority Code,Calendar Year,LA CO2 Sector,LA CO2 Sub-sector,Territorial emissions (kt CO2),Emissions within the scope of influence of LAs (kt CO2),Mid-year Population (thousands),Area (km2)
0,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Commercial,Commercial Electricity,77.193230,77.193230,90.457,98.3466
1,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Commercial,Commercial Gas,16.689241,16.689241,90.457,98.3466
2,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Commercial,Commercial 'Other Fuels',0.198111,0.198111,90.457,98.3466
3,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Domestic,Domestic Electricity,80.639845,80.639845,90.457,98.3466
4,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Domestic,Domestic Gas,131.383713,131.383713,90.457,98.3466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106989,England,E92000001,London,E12000007,Westminster,Westminster,E09000033,2019,Public Sector,Public Sector 'Other Fuels',0.857591,0.857591,261.317,22.0301
106990,England,E92000001,London,E12000007,Westminster,Westminster,E09000033,2019,Transport,Road Transport (A roads),180.856113,180.856113,261.317,22.0301
106991,England,E92000001,London,E12000007,Westminster,Westminster,E09000033,2019,Transport,Road Transport (Minor roads),89.550258,89.550258,261.317,22.0301
106992,England,E92000001,London,E12000007,Westminster,Westminster,E09000033,2019,Transport,Diesel Railways,1.992724,0.000000,261.317,22.0301


In [123]:
Northern_Ireland =my_data[my_data['Country']=='Northern Ireland']#Extracting England data from the dataframe
Northern_Ireland

,Country,Country Code,Region,Region Code,Second Tier Authority,Local Authority,Local Authority Code,Calendar Year,LA CO2 Sector,LA CO2 Sub-sector,Territorial emissions (kt CO2),Emissions within the scope of influence of LAs (kt CO2),Mid-year Population (thousands),Area (km2)
106994,Northern Ireland,N92000002,Northern Ireland,N92000002,Northern Ireland,Antrim and Newtownabbey,N09000001,2005,Commercial,Commercial Electricity,48.606519,48.606519,132.350,728.1864
106995,Northern Ireland,N92000002,Northern Ireland,N92000002,Northern Ireland,Antrim and Newtownabbey,N09000001,2005,Commercial,Commercial Gas,16.256937,16.256937,132.350,728.1864
106996,Northern Ireland,N92000002,Northern Ireland,N92000002,Northern Ireland,Antrim and Newtownabbey,N09000001,2005,Commercial,Commercial 'Other Fuels',3.548631,3.548631,132.350,728.1864
106997,Northern Ireland,N92000002,Northern Ireland,N92000002,Northern Ireland,Antrim and Newtownabbey,N09000001,2005,Domestic,Domestic Electricity,128.801606,128.801606,132.350,728.1864
106998,Northern Ireland,N92000002,Northern Ireland,N92000002,Northern Ireland,Antrim and Newtownabbey,N09000001,2005,Domestic,Domestic Gas,16.966301,16.966301,132.350,728.1864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110668,Northern Ireland,N92000002,Northern Ireland,N92000002,Northern Ireland,Ards and North Down,N09000011,2019,Public Sector,Public Sector 'Other Fuels',0.838211,0.838211,161.725,566.4732
110669,Northern Ireland,N92000002,Northern Ireland,N92000002,Northern Ireland,Ards and North Down,N09000011,2019,Transport,Road Transport (A roads),118.966924,118.966924,161.725,566.4732
110670,Northern Ireland,N92000002,Northern Ireland,N92000002,Northern Ireland,Ards and North Down,N09000011,2019,Transport,Road Transport (Minor roads),65.737584,65.737584,161.725,566.4732
110671,Northern Ireland,N92000002,Northern Ireland,N92000002,Northern Ireland,Ards and North Down,N09000011,2019,Transport,Diesel Railways,4.334469,0.000000,161.725,566.4732


In [68]:
Scot_land =my_data[my_data['Country']=='Scotland']#Extracting England data from the dataframe
Scot_land

,Country,Country Code,Region,Region Code,Second Tier Authority,Local Authority,Local Authority Code,Calendar Year,LA CO2 Sector,LA CO2 Sub-sector,Territorial emissions (kt CO2),Emissions within the scope of influence of LAs (kt CO2),Mid-year Population (thousands),Area (km2)
110703,Scotland,S92000003,Scotland,S92000003,Scotland,Clackmannanshire,S12000005,2005,Commercial,Commercial Electricity,41.268053,41.268053,49.16,163.9126
110704,Scotland,S92000003,Scotland,S92000003,Scotland,Clackmannanshire,S12000005,2005,Commercial,Commercial Gas,29.402851,29.402851,49.16,163.9126
110705,Scotland,S92000003,Scotland,S92000003,Scotland,Clackmannanshire,S12000005,2005,Commercial,Commercial 'Other Fuels',0.163952,0.163952,49.16,163.9126
110706,Scotland,S92000003,Scotland,S92000003,Scotland,Clackmannanshire,S12000005,2005,Domestic,Domestic Electricity,54.052582,54.052582,49.16,163.9126
110707,Scotland,S92000003,Scotland,S92000003,Scotland,Clackmannanshire,S12000005,2005,Domestic,Domestic Gas,72.955597,72.955597,49.16,163.9126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121626,Scotland,S92000003,Scotland,S92000003,Scotland,North Lanarkshire,S12000050,2019,Transport,Road Transport (A roads),182.256170,182.256170,341.37,472.3091
121627,Scotland,S92000003,Scotland,S92000003,Scotland,North Lanarkshire,S12000050,2019,Transport,Road Transport (Motorways),305.541643,0.000000,341.37,472.3091
121628,Scotland,S92000003,Scotland,S92000003,Scotland,North Lanarkshire,S12000050,2019,Transport,Road Transport (Minor roads),281.302384,281.302384,341.37,472.3091
121629,Scotland,S92000003,Scotland,S92000003,Scotland,North Lanarkshire,S12000050,2019,Transport,Diesel Railways,7.612655,0.000000,341.37,472.3091


In [69]:
Wales1 =my_data[my_data['Country']=='Wales']#Extracting England data from the dataframe
Wales1

,Country,Country Code,Region,Region Code,Second Tier Authority,Local Authority,Local Authority Code,Calendar Year,LA CO2 Sector,LA CO2 Sub-sector,Territorial emissions (kt CO2),Emissions within the scope of influence of LAs (kt CO2),Mid-year Population (thousands),Area (km2)
121631,Wales,W92000004,Wales,W92000004,Wales,Isle of Anglesey,W06000001,2005,Commercial,Commercial Electricity,47.030652,47.030652,69.095,749.4385
121632,Wales,W92000004,Wales,W92000004,Wales,Isle of Anglesey,W06000001,2005,Commercial,Commercial Gas,11.552509,11.552509,69.095,749.4385
121633,Wales,W92000004,Wales,W92000004,Wales,Isle of Anglesey,W06000001,2005,Commercial,Commercial 'Other Fuels',1.138490,1.138490,69.095,749.4385
121634,Wales,W92000004,Wales,W92000004,Wales,Isle of Anglesey,W06000001,2005,Domestic,Domestic Electricity,93.465801,93.465801,69.095,749.4385
121635,Wales,W92000004,Wales,W92000004,Wales,Isle of Anglesey,W06000001,2005,Domestic,Domestic Gas,46.755134,46.755134,69.095,749.4385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129260,Wales,W92000004,Wales,W92000004,Wales,Merthyr Tydfil,W06000024,2019,Public Sector,Public Sector 'Other Fuels',0.105123,0.105123,60.326,111.9569
129261,Wales,W92000004,Wales,W92000004,Wales,Merthyr Tydfil,W06000024,2019,Transport,Road Transport (A roads),58.731503,58.731503,60.326,111.9569
129262,Wales,W92000004,Wales,W92000004,Wales,Merthyr Tydfil,W06000024,2019,Transport,Road Transport (Minor roads),23.451077,23.451077,60.326,111.9569
129263,Wales,W92000004,Wales,W92000004,Wales,Merthyr Tydfil,W06000024,2019,Transport,Diesel Railways,2.652905,0.000000,60.326,111.9569


In [42]:
my_data = my_data[my_data.isna().any(axis=1)]
print('\n These are the missing data \n', my_data)


 These are the missing data 
        Country Country Code Region Region Code Second Tier Authority  \
110673     NaN          NaN    NaN         NaN                   NaN   
110674     NaN          NaN    NaN         NaN                   NaN   
110675     NaN          NaN    NaN         NaN                   NaN   
110676     NaN          NaN    NaN         NaN                   NaN   
110677     NaN          NaN    NaN         NaN                   NaN   
...        ...          ...    ...         ...                   ...   
129383     NaN          NaN    NaN         NaN                   NaN   
129384     NaN          NaN    NaN         NaN                   NaN   
129385     NaN          NaN    NaN         NaN                   NaN   
129386     NaN          NaN    NaN         NaN                   NaN   
129387     NaN          NaN    NaN         NaN                   NaN   

                   Local Authority Local Authority Code  Calendar Year  \
110673  Unallocated electricit

In [53]:
my_data.describe()

,Calendar Year,Territorial emissions (kt CO2),Emissions within the scope of influence of LAs (kt CO2),Mid-year Population (thousands),Area (km2)
count,129388.000000,129388.000000,129388.000000,129235.000000,129235.000000
mean,2012.004575,51.333002,42.798656,169.407561,660.715411
std,4.321080,169.687627,83.154397,114.484123,1637.797146
min,2005.000000,-3256.716149,0.000000,2.210000,3.149600
25%,2008.000000,1.670336,0.000000,97.605000,97.958400
50%,2012.000000,15.379282,8.340660,134.934000,275.623400
75%,2016.000000,65.511506,54.758404,208.182000,661.060800
max,2019.000000,9529.019680,4025.260014,1141.816000,26473.949600


,Country,Country Code,Region,Region Code,Second Tier Authority,Local Authority,Local Authority Code,Calendar Year,LA CO2 Sector,LA CO2 Sub-sector,CO2_emission,Emissions within the scope of influence of LAs (kt CO2),Mid-year Population (thousands),Area (km2)
0,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Commercial,Commercial Electricity,7.719323e+01,77.193230,90.457,98.3466
1,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Commercial,Commercial Gas,1.668924e+01,16.689241,90.457,98.3466
2,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Commercial,Commercial 'Other Fuels',1.981106e-01,0.198111,90.457,98.3466
3,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Domestic,Domestic Electricity,8.063985e+01,80.639845,90.457,98.3466
4,England,E92000001,North East,E12000001,Hartlepool,Hartlepool,E06000001,2005,Domestic,Domestic Gas,1.313837e+02,131.383713,90.457,98.3466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129383,NaN,NaN,NaN,NaN,NaN,Unallocated consumption,NaN,2019,Industry,Industry Gas,2.574098e+02,257.409829,NaN,NaN
129384,NaN,NaN,NaN,NaN,NaN,Unallocated consumption,NaN,2019,LULUCF,LULUCF Net Emissions: Forest land,-3.270000e-11,0.000000,NaN,NaN
129385,NaN,NaN,NaN,NaN,NaN,Unallocated consumption,NaN,2019,LULUCF,LULUCF Net Emissions: Cropland,1.820000e-11,0.000000,NaN,NaN
129386,NaN,NaN,NaN,NaN,NaN,Unallocated consumption,NaN,2019,LULUCF,LULUCF Net Emissions: Grassland,8.640000e-12,0.000000,NaN,NaN


In [111]:
# btc=new_data[new_data['symbol']=='BTC']#Extracting the bitcoin data from the dataframe
Eng_land= Eng_land[['Calendar Year', 'Territorial emissions (kt CO2)']]
Eng_land
#Eng_land.drop
# btc.drop(['symbol','name'],axis=1,inplace=True)#Just dropping columns here

,Calendar Year,Territorial emissions (kt CO2)
0,2005,77.193230
1,2005,16.689241
2,2005,0.198111
3,2005,80.639845
4,2005,131.383713
...,...,...
106989,2019,0.857591
106990,2019,180.856113
106991,2019,89.550258
106992,2019,1.992724


In [106]:
# Eng_land['Calendar Year'] = pd.to_datetime(Eng_land['Calendar Year'])

In [112]:
 Eng_land = Eng_land.set_index('Calendar Year')
Eng_land

,Territorial emissions (kt CO2)
Calendar Year,
2005,77.193230
2005,16.689241
2005,0.198111
2005,80.639845
2005,131.383713
...,...
2019,0.857591
2019,180.856113
2019,89.550258


In [86]:
Eng_land.rename(columns={'Territorial emissions (kt CO2)':'CO2_emission'})

,CO2_emission
Calendar Year,
2005,77.193230
2005,16.689241
2005,0.198111
2005,80.639845
2005,131.383713
...,...
2019,0.857591
2019,180.856113
2019,89.550258


KeyError: 'Calendar Year'

In [113]:
plt.figure(figsize=(14,7))
fig=plt.figure(1)
ax1= fig.add_subplot(111)
ax1.set_xlabel('Time frame')
ax1.set_ylabel('CO2 emissions')
ax1.set_title('Close price for BTC')
ax1.plot(Eng_land.Territorial emissions (kt CO2))

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3610253606.py, line 7)

In [124]:
import matplotlib

%matplotlib notebook
plt.plot (Eng_land, label= England)
plt.plot(Northern_Ireland, label= Northern)
plt.xlabel('Date')
plt.grid()
plt.legend

<IPython.core.display.Javascript object>

TypeError: unhashable type: 'numpy.ndarray'